# 책임 있는 AI 대시보드를 만들어 모델 평가

기계 학습 모델을 비교하고 평가할 때 성능 메트릭 이상을 검토하려고 합니다. Azure Machine Learning을 사용하면 책임 있는 AI 대시보드를 만들어 모델의 다양한 데이터 코호트에서 수행되는 방식을 탐색할 수 있습니다. 

## 데이터 준비

책임 있는 AI 대시보드를 만들려면 Parquet 파일로 저장되고 데이터 자산으로 등록된 학습 및 테스트 데이터 세트가 필요합니다.

데이터는 현재 CSV 파일로 저장됩니다. Parquet 파일로 변환해 보겠습니다. 


In [ ]:
## 계속하기 전에

이 Notebook에서 코드를 실행하려면 **최신 버전의 azureml-ai-ml** 패키지가 필요합니다. 아래 셀의 명령을 실행하여 이 패키지가 설치되어 있는지 확인합니다.

> **고**:
> **azure-ai-ml** 패키지가 설치되지 않은 경우 를 실행 `pip install azure-ai-ml` 하여 설치합니다.

In [ ]:
## 작업 영역에 연결

필요한 SDK 패키지를 설치했으므로 작업 영역에 연결할 수 있습니다.

작업 영역에 연결하려면 식별자 매개 변수(구독 ID, 리소스 그룹 이름 및 작업 영역 이름)가 필요합니다. 리소스 그룹 이름 및 작업 영역 이름이 이미 채워져 있습니다. 명령을 완료하려면 구독 ID만 필요합니다.

필요한 매개 변수를 찾으려면 Studio의 오른쪽 위에 있는 구독 및 작업 영역 이름을 클릭합니다. 오른쪽에 창이 열립니다.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 구독 ID를 복사하고 **YOUR-SUBSCRIPTION-ID** 를 복사한 값으로 바꿉니다. </p>

## 데이터 자산 만들기

책임 있는 AI 대시보드를 만들려면 학습 및 테스트 데이터 세트를 **MLtable** 데이터 자산으로 등록해야 합니다. MLtable 데이터 자산은 이전에 만든 Parquet 파일을 참조합니다.

In [ ]:
## 파이프라인을 빌드하여 책임 있는 AI 대시보드 만들기

대시보드를 만들려면 Azure Machine Learning 작업 영역에 기본적으로 등록된 미리 빌드된 구성 요소를 사용하여 파이프라인을 빌드합니다.

### 모델 등록

기계 학습 모델이 이미 학습되었습니다. 이 모델은 환자에게 당뇨병이 있는지 여부를 예측합니다. 모든 모델 파일은 폴더에 `model` 저장됩니다. 

폴더 및 해당 내용을 가리켜 `model` 모델을 등록합니다.

In [ ]:
## 파이프라인 빌드

책임 있는 AI 대시보드를 만들려면 미리 빌드된 구성 요소를 사용하여 파이프라인을 만듭니다. 사용할 구성 요소와 대시보드에 포함할 기능을 선택할 수 있습니다. 오류 분석 및 모델 해석성을 포함하는 대시보드를 만듭니다. 

책임 있는 AI 기능 옆에 대시보드를 빌드하는 파이프라인에는 대시보드를 생성하기 위한 시작 부분에 구성 요소와 생성된 모든 인사이트를 수집하기 위한 구성 요소가 포함되어야 합니다.

In [ ]:
사용하려는 모든 구성 요소를 검색한 경우 파이프라인을 빌드하고 구성 요소를 적절한 순서로 연결할 수 있습니다.

1. 대시보드를 생성합니다.
1. 오류 분석을 추가합니다.
1. 설명을 추가합니다.
1. 모든 인사이트를 수집하고 대시보드에서 시각화합니다.

이제 파이프라인이 빌드되었으므로 학습 및 테스트 데이터 세트라는 두 가지 필수 입력을 정의해야 합니다.

In [ ]:
마지막으로 파이프라인에 입력을 할당하고 대상 열(예측 레이블)을 설정하는 모든 항목을 함께 배치합니다.

In [ ]:
## 파이프라인 실행

파이프라인을 성공적으로 빌드한 경우 제출할 수 있습니다. 다음 코드는 파이프라인을 제출하고 파이프라인의 상태를 확인합니다. 스튜디오에서 파이프라인의 상태를 볼 수도 있습니다. 

파이프라인이 완료되면 스튜디오에서 대시보드를 검토할 수 있습니다. 

In [ ]:
# Get handle to azureml registry for the RAI built in components
registry_name = "azureml"
ml_client_registry = MLClient(
    credential=credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    registry_name=registry_name,
)
print(ml_client_registry)

### Register the model

A machine learning model has already been trained for you. The model predicts whether a patient has diabetes. All model files are stored in the `model` folder. 

Register the model by pointing to the `model` folder and its contents.

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

file_model = Model(
    path="model",
    type=AssetTypes.MLFLOW_MODEL,
    name="local-mlflow-diabetes",
    description="Model created from local file.",
)
model = ml_client.models.create_or_update(file_model)

## Build the pipeline

To create the responsible AI dashboard, you'll create a pipeline using the prebuilt components. You can choose which components to use, and which features you want to include in your dashboard. You'll create a dashboard that includes error analysis and model interpretability. 

Next to the responsible AI features, a pipeline to build a dashboard needs to include a component at the start to construct the dashboard, and a component at the end to gather all generated insights.

In [ ]:
model_name = model.name
expected_model_id = f"{model_name}:1"
azureml_model_id = f"azureml:{expected_model_id}"

In [ ]:
label = "latest"

rai_constructor_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_insight_constructor", label=label
)

# we get latest version and use the same version for all components
version = rai_constructor_component.version
print("The current version of RAI built-in components is: " + version)

rai_erroranalysis_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_erroranalysis", version=version
)

rai_explanation_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_explanation", version=version
)

rai_gather_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_insight_gather", version=version
)

When you've retrieved all components you want to use, you can build the pipeline and connect the components in the appropriate order:

1. Construct the dashboard.
1. Add error analysis.
1. Add explanations.
1. Gather all insights and visualize them in the dashboard.

In [ ]:
from azure.ai.ml import Input, dsl
from azure.ai.ml.constants import AssetTypes

compute_name = "aml-cluster"

@dsl.pipeline(
    compute=compute_name,
    description="RAI insights on diabetes data",
    experiment_name=f"RAI_insights_{model_name}",
)
def rai_decision_pipeline(
    target_column_name, train_data, test_data
):
    # Initiate the RAIInsights
    create_rai_job = rai_constructor_component(
        title="RAI dashboard diabetes",
        task_type="classification",
        model_info=expected_model_id,
        model_input=Input(type=AssetTypes.MLFLOW_MODEL, path=azureml_model_id),
        train_dataset=train_data,
        test_dataset=test_data,
        target_column_name=target_column_name,
    )
    create_rai_job.set_limits(timeout=30)

    # Add error analysis
    error_job = rai_erroranalysis_component(
        rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,
    )
    error_job.set_limits(timeout=10)

    # Add explanations
    explanation_job = rai_explanation_component(
        rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,
        comment="add explanation", 
    )
    explanation_job.set_limits(timeout=10)

    # Combine everything
    rai_gather_job = rai_gather_component(
        constructor=create_rai_job.outputs.rai_insights_dashboard,
        insight_3=error_job.outputs.error_analysis,
        insight_4=explanation_job.outputs.explanation,
    )
    rai_gather_job.set_limits(timeout=10)

    rai_gather_job.outputs.dashboard.mode = "upload"

    return {
        "dashboard": rai_gather_job.outputs.dashboard,
    }

Now the pipeline has been built, you need to define the two necessary inputs: the training and test dataset.

In [ ]:
from azure.ai.ml import Input
target_feature = "Diabetic"

diabetes_train_pq = Input(
    type="mltable",
    path=f"azureml:{input_train_data}:{data_version}",
    mode="download",
)
diabetes_test_pq = Input(
    type="mltable",
    path=f"azureml:{input_test_data}:{data_version}",
    mode="download",
)

Finally, we'll put everything together: assign the inputs to the pipeline and set the target column (the predicted label).

In [ ]:
import uuid
from azure.ai.ml import Output

# Pipeline to construct the RAI Insights
insights_pipeline_job = rai_decision_pipeline(
    target_column_name="Diabetic",
    train_data=diabetes_train_pq,
    test_data=diabetes_test_pq,
)

# Workaround to enable the download
rand_path = str(uuid.uuid4())
insights_pipeline_job.outputs.dashboard = Output(
    path=f"azureml://datastores/workspaceblobstore/paths/{rand_path}/dashboard/",
    mode="upload",
    type="uri_folder",
)


## Run the pipeline

When you've successfully built the pipeline, you can submit it. The following code will submit the pipeline and check the status of the pipeline. You can also view the pipeline's status in the studio. 

In [ ]:
from azure.ai.ml.entities import PipelineJob
from IPython.core.display import HTML
from IPython.display import display
import time

def submit_and_wait(ml_client, pipeline_job) -> PipelineJob:
    created_job = ml_client.jobs.create_or_update(pipeline_job)
    assert created_job is not None

    print("Pipeline job can be accessed in the following URL:")
    display(HTML('<a href="{0}">{0}</a>'.format(created_job.studio_url)))

    while created_job.status not in [
        "Completed",
        "Failed",
        "Canceled",
        "NotResponding",
    ]:
        time.sleep(30)
        created_job = ml_client.jobs.get(created_job.name)
        print("Latest status : {0}".format(created_job.status))
    assert created_job.status == "Completed"
    return created_job


# This is the actual submission
insights_job = submit_and_wait(ml_client, insights_pipeline_job)

When the pipeline is completed, you can review the dashboard in the studio. 